# Multivariate Volatility Models

This notebook demonstrates the use of multivariate volatility models in the MFE Toolbox. We'll cover:

1. Introduction to multivariate volatility modeling
2. Data preparation and exploration for multiple assets
3. Correlation analysis and visualization
4. Dynamic Conditional Correlation (DCC) model estimation
5. BEKK model for covariance dynamics
6. Constant Conditional Correlation (CCC) model
7. Other multivariate models (OGARCH, RARCH, etc.)
8. Portfolio volatility estimation and forecasting
9. Asynchronous processing for computationally intensive models

The MFE Toolbox provides a comprehensive set of multivariate volatility models for analyzing multiple asset returns, implemented using modern Python practices with NumPy, Pandas, and Numba acceleration.

## 1. Setup and Imports

First, let's import the necessary modules and set up our environment.

In [ ]:
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple, Optional, Union, Any
import asyncio
import datetime
import warnings

# MFE Toolbox imports
import mfe
from mfe.models.multivariate import DCC, BEKK, CCC, OGARCH, RARCH, RCC, RiskMetrics
from mfe.models.univariate import GARCH
from mfe.models.distributions import Normal, StudentT, GED, SkewedT
from mfe.utils.data_transformations import returns_from_prices
from mfe.utils.matrix_ops import cov2corr

# Configure plotting
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context("notebook", font_scale=1.2)
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['figure.dpi'] = 100

# Display version information
print(f"MFE Toolbox version: {mfe.__version__}")

## 2. Data Preparation and Exploration for Multiple Assets

We'll use financial market data for multiple assets to demonstrate multivariate volatility modeling.

In [ ]:
# Load sample data for multiple assets
# In a real application, you might use yfinance, pandas-datareader, or your own data source
# For this example, we'll create a function to download data using pandas-datareader

def load_market_data(tickers: List[str], 
                     start_date: str = "2010-01-01", 
                     end_date: Optional[str] = None) -> pd.DataFrame:
    """
    Load market data for multiple tickers.
    
    Parameters
    ----------
    tickers : List[str]
        List of ticker symbols to load
    start_date : str
        Start date in YYYY-MM-DD format
    end_date : str, optional
        End date in YYYY-MM-DD format, defaults to current date
        
    Returns
    -------
    pd.DataFrame
        DataFrame containing adjusted close prices for all tickers
    """
    try:
        import pandas_datareader as pdr
        
        # Set end date to today if not provided
        if end_date is None:
            end_date = datetime.datetime.now().strftime("%Y-%m-%d")
            
        # Initialize DataFrame to store results
        all_data = pd.DataFrame()
        
        # Download data for each ticker
        for ticker in tickers:
            try:
                data = pdr.get_data_yahoo(ticker, start=start_date, end=end_date)
                all_data[ticker] = data['Adj Close']
                print(f"Downloaded data for {ticker}: {len(data)} days")
            except Exception as e:
                print(f"Error downloading data for {ticker}: {e}")
        
        return all_data
    except ImportError:
        print("pandas-datareader not installed. Please install with: pip install pandas-datareader")
        # Create synthetic data as fallback
        return create_synthetic_market_data(tickers, start_date, end_date)

def create_synthetic_market_data(tickers: List[str], 
                                start_date: str = "2010-01-01", 
                                end_date: Optional[str] = None) -> pd.DataFrame:
    """
    Create synthetic market data for multiple assets.
    
    Parameters
    ----------
    tickers : List[str]
        List of ticker symbols to simulate
    start_date : str
        Start date in YYYY-MM-DD format
    end_date : str, optional
        End date in YYYY-MM-DD format, defaults to current date
        
    Returns
    -------
    pd.DataFrame
        DataFrame containing synthetic price data for all tickers
    """
    # Parse dates
    start = pd.to_datetime(start_date)
    if end_date is None:
        end = pd.to_datetime(datetime.datetime.now().strftime("%Y-%m-%d"))
    else:
        end = pd.to_datetime(end_date)
    
    # Create date range (business days only)
    dates = pd.date_range(start=start, end=end, freq='B')
    
    # Set random seed for reproducibility
    np.random.seed(42)
    
    # Number of assets
    n_assets = len(tickers)
    n_days = len(dates)
    
    # Create correlation matrix with realistic correlations
    # We'll use a factor model approach to ensure positive definiteness
    # First, create random loadings on common factors
    n_factors = 3  # Market, size, value factors for example
    factor_loadings = np.random.uniform(0.3, 0.9, size=(n_assets, n_factors))
    
    # Create correlation matrix from factor loadings
    corr_matrix = factor_loadings @ factor_loadings.T
    # Add idiosyncratic variance to ensure diagonal of 1s
    for i in range(n_assets):
        corr_matrix[i, i] = 1.0
    
    # Ensure it's a valid correlation matrix
    # Make it symmetric
    corr_matrix = (corr_matrix + corr_matrix.T) / 2
    # Normalize to ensure diagonal is 1
    d = np.sqrt(np.diag(corr_matrix))
    corr_matrix = corr_matrix / np.outer(d, d)
    
    # Create volatilities for each asset (annualized)
    asset_vols = np.random.uniform(0.15, 0.35, size=n_assets)  # 15% to 35% annual vol
    
    # Convert to daily volatility
    daily_vols = asset_vols / np.sqrt(252)
    
    # Create covariance matrix
    cov_matrix = np.diag(daily_vols) @ corr_matrix @ np.diag(daily_vols)
    
    # Generate correlated returns
    # We'll use Cholesky decomposition
    chol = np.linalg.cholesky(cov_matrix)
    
    # Generate random normal returns
    random_returns = np.random.normal(0, 1, size=(n_days, n_assets))
    
    # Transform to correlated returns
    correlated_returns = random_returns @ chol.T
    
    # Add drift (expected return)
    expected_returns = np.random.uniform(0.05, 0.15, size=n_assets) / 252  # 5% to 15% annual return
    correlated_returns += expected_returns
    
    # Add volatility clustering
    # We'll use a simple AR(1) process for volatility
    vol_persistence = 0.95
    vol_scale = np.ones((n_days, n_assets))
    vol_innovation = np.random.normal(0, 0.1, size=(n_days, n_assets))
    
    for t in range(1, n_days):
        vol_scale[t] = np.sqrt(0.05 + vol_persistence * vol_scale[t-1]**2 + 0.05 * vol_innovation[t]**2)
    
    # Apply time-varying volatility
    correlated_returns = correlated_returns * vol_scale
    
    # Convert returns to prices
    # Start with price of 100 for each asset
    prices = np.zeros((n_days, n_assets))
    prices[0] = 100.0
    
    for t in range(1, n_days):
        prices[t] = prices[t-1] * (1 + correlated_returns[t])
    
    # Create DataFrame
    price_df = pd.DataFrame(prices, index=dates, columns=tickers)
    
    print(f"Created synthetic data for {n_assets} assets over {n_days} days")
    return price_df

# Define tickers to analyze
# We'll use major US indices and some sector ETFs
tickers = ['SPY', 'QQQ', 'IWM', 'XLF', 'XLE', 'XLK', 'XLV', 'XLI']

# Load data
try:
    price_data = load_market_data(tickers, "2015-01-01")
    print(f"Loaded data for {len(price_data.columns)} assets over {len(price_data)} days")
except Exception as e:
    print(f"Error loading data: {e}")
    print("Using synthetic data instead")
    price_data = create_synthetic_market_data(tickers, "2015-01-01")

# Display the first few rows
price_data.head()

In [ ]:
# Check for missing values
missing_values = price_data.isna().sum()
print("Missing values per asset:")
print(missing_values)

# Handle missing values if necessary
if missing_values.sum() > 0:
    print("
Handling missing values...")
    # Forward fill missing values
    price_data = price_data.fillna(method='ffill')
    # Backward fill any remaining missing values (at the beginning)
    price_data = price_data.fillna(method='bfill')
    print("Missing values after handling:", price_data.isna().sum().sum())

# Calculate returns
# We'll use log returns for volatility modeling
returns_df = pd.DataFrame()
for ticker in price_data.columns:
    returns_df[ticker] = returns_from_prices(price_data[ticker], log=True) * 100  # Convert to percentage

# Display summary statistics
returns_summary = returns_df.describe()
print("
Returns Summary Statistics:")
returns_summary

In [ ]:
# Plot price series
plt.figure(figsize=(14, 8))

# Normalize prices to start at 100 for easier comparison
normalized_prices = price_data.div(price_data.iloc[0]) * 100

for ticker in normalized_prices.columns:
    plt.plot(normalized_prices.index, normalized_prices[ticker], label=ticker)

plt.title('Normalized Price Series (Base = 100)')
plt.ylabel('Normalized Price')
plt.xlabel('Date')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Plot returns
fig, axes = plt.subplots(len(returns_df.columns), 1, figsize=(14, 3*len(returns_df.columns)), sharex=True)

for i, ticker in enumerate(returns_df.columns):
    axes[i].plot(returns_df.index, returns_df[ticker], label=ticker)
    axes[i].set_title(f'{ticker} Daily Returns (%)')
    axes[i].set_ylabel('Returns (%)')
    axes[i].legend(loc='upper right')
    axes[i].grid(True)

axes[-1].set_xlabel('Date')
plt.tight_layout()
plt.show()

## 3. Correlation Analysis and Visualization

Let's analyze the correlation structure between the assets.

In [ ]:
# Calculate unconditional correlation matrix
corr_matrix = returns_df.corr()

# Plot correlation matrix as a heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0,
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('Unconditional Correlation Matrix')
plt.tight_layout()
plt.show()

In [ ]:
# Calculate rolling correlations to see how they evolve over time
# We'll focus on a few key pairs
window_size = 60  # 60-day rolling window (approximately 3 months)

# Define pairs to analyze
pairs = [
    ('SPY', 'QQQ'),   # S&P 500 vs NASDAQ
    ('SPY', 'IWM'),   # S&P 500 vs Russell 2000
    ('SPY', 'XLF'),   # S&P 500 vs Financials
    ('SPY', 'XLE'),   # S&P 500 vs Energy
    ('XLF', 'XLE'),   # Financials vs Energy
    ('XLK', 'XLV')    # Technology vs Healthcare
]

# Calculate rolling correlations
rolling_corrs = {}
for asset1, asset2 in pairs:
    rolling_corrs[(asset1, asset2)] = returns_df[asset1].rolling(window=window_size).corr(returns_df[asset2])

# Plot rolling correlations
plt.figure(figsize=(14, 8))

for pair, corr in rolling_corrs.items():
    plt.plot(corr.index, corr, label=f'{pair[0]} vs {pair[1]}')

plt.title(f'{window_size}-Day Rolling Correlations')
plt.ylabel('Correlation')
plt.xlabel('Date')
plt.legend()
plt.grid(True)
plt.ylim(-0.2, 1.0)  # Adjust as needed
plt.show()

In [ ]:
# Calculate rolling volatilities
rolling_vols = returns_df.rolling(window=window_size).std() * np.sqrt(252)  # Annualized

# Plot rolling volatilities
plt.figure(figsize=(14, 8))

for ticker in rolling_vols.columns:
    plt.plot(rolling_vols.index, rolling_vols[ticker], label=ticker)

plt.title(f'{window_size}-Day Rolling Annualized Volatility')
plt.ylabel('Volatility (%)')
plt.xlabel('Date')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Analyze the distribution of returns
# We'll create a pairplot for a subset of assets
subset_tickers = ['SPY', 'QQQ', 'XLF', 'XLE']  # Subset for clarity
subset_returns = returns_df[subset_tickers]

# Create pairplot
sns.pairplot(subset_returns, diag_kind='kde', plot_kws={'alpha': 0.6})
plt.suptitle('Pairwise Return Distributions and Scatter Plots', y=1.02, fontsize=16)
plt.tight_layout()
plt.show()

## 4. Dynamic Conditional Correlation (DCC) Model Estimation

Now let's estimate a DCC model to capture time-varying correlations between assets.

In [ ]:
# Prepare data for DCC estimation
# Convert returns DataFrame to NumPy array
returns_array = returns_df.values

# Create and estimate a DCC-GARCH model
# First, we'll use normal innovations for simplicity
dcc_model = DCC(returns_df.shape[1],  # Number of assets
                p=1, q=1,             # DCC orders
                univariate_model=GARCH,  # Univariate model for each series
                univariate_params={'p': 1, 'q': 1},  # GARCH(1,1) for each series
                distribution=Normal())

# Fit the model
print("Estimating DCC-GARCH model...")
dcc_results = dcc_model.fit(returns_array)
print("DCC-GARCH estimation complete.")

# Display estimation results
print("
DCC-GARCH Estimation Results:")
print(f"Log-Likelihood: {dcc_results.log_likelihood:.4f}")
print(f"AIC: {dcc_results.aic:.4f}")
print(f"BIC: {dcc_results.bic:.4f}")

# Display DCC parameters
print("
DCC Parameters:")
dcc_params = pd.DataFrame({
    'Parameter': dcc_results.parameter_names[-2:],  # Last two parameters are DCC parameters
    'Estimate': dcc_results.parameters[-2:],
    'Std. Error': dcc_results.std_errors[-2:],
    't-statistic': dcc_results.t_stats[-2:],
    'p-value': dcc_results.p_values[-2:]
})
print(dcc_params)

# Calculate persistence
alpha = dcc_results.parameters[-2]  # DCC alpha parameter
beta = dcc_results.parameters[-1]   # DCC beta parameter
persistence = alpha + beta
print(f"
DCC Persistence (α + β): {persistence:.4f}")
print(f"Half-life: {np.log(0.5) / np.log(persistence):.2f} days")

In [ ]:
# Extract conditional correlations from DCC model
conditional_correlations = dcc_results.conditional_correlations

# Create a dictionary to store time series of pairwise correlations
pairwise_correlations = {}
for i, asset1 in enumerate(returns_df.columns):
    for j, asset2 in enumerate(returns_df.columns):
        if i < j:  # Only store unique pairs
            pair = (asset1, asset2)
            pairwise_correlations[pair] = [corr_matrix[i, j] for corr_matrix in conditional_correlations]

# Convert to DataFrame for easier plotting
corr_df = pd.DataFrame(pairwise_correlations, index=returns_df.index)

# Plot conditional correlations for selected pairs
plt.figure(figsize=(14, 8))

for pair in pairs:
    if pair in corr_df.columns:
        plt.plot(corr_df.index, corr_df[pair], label=f'{pair[0]} vs {pair[1]}')
    elif (pair[1], pair[0]) in corr_df.columns:  # Check reverse order
        plt.plot(corr_df.index, corr_df[(pair[1], pair[0])], label=f'{pair[0]} vs {pair[1]}')

plt.title('DCC-GARCH Conditional Correlations')
plt.ylabel('Correlation')
plt.xlabel('Date')
plt.legend()
plt.grid(True)
plt.ylim(-0.2, 1.0)  # Adjust as needed
plt.show()

In [ ]:
# Compare DCC conditional correlations with rolling correlations
plt.figure(figsize=(14, 8))

# Select a specific pair for comparison
comparison_pair = ('SPY', 'XLF')  # S&P 500 vs Financials

# Get DCC correlation for this pair
if comparison_pair in corr_df.columns:
    dcc_corr = corr_df[comparison_pair]
elif (comparison_pair[1], comparison_pair[0]) in corr_df.columns:  # Check reverse order
    dcc_corr = corr_df[(comparison_pair[1], comparison_pair[0])]
else:
    dcc_corr = None

# Get rolling correlation for this pair
rolling_corr = rolling_corrs.get(comparison_pair)
if rolling_corr is None:
    rolling_corr = rolling_corrs.get((comparison_pair[1], comparison_pair[0]))

# Plot both correlations
if dcc_corr is not None:
    plt.plot(dcc_corr.index, dcc_corr, label='DCC-GARCH', linewidth=2)
if rolling_corr is not None:
    plt.plot(rolling_corr.index, rolling_corr, label=f'{window_size}-Day Rolling', linewidth=2, alpha=0.7)

plt.title(f'Comparison of Correlation Estimates: {comparison_pair[0]} vs {comparison_pair[1]}')
plt.ylabel('Correlation')
plt.xlabel('Date')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Visualize the evolution of the full correlation matrix over time
# We'll create a heatmap animation for selected dates

# Select dates for visualization (e.g., every 6 months)
date_indices = np.linspace(0, len(returns_df) - 1, 6, dtype=int)
selected_dates = returns_df.index[date_indices]

# Create a figure with subplots for each date
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.flatten()

for i, date_idx in enumerate(date_indices):
    date = returns_df.index[date_idx]
    corr_matrix = conditional_correlations[date_idx]
    
    # Create heatmap
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0,
                square=True, linewidths=0.5, cbar=False, ax=axes[i],
                xticklabels=returns_df.columns, yticklabels=returns_df.columns)
    axes[i].set_title(f'Conditional Correlation Matrix: {date.strftime("%Y-%m-%d")}')

# Add a colorbar to the figure
cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
sm = plt.cm.ScalarMappable(cmap='coolwarm', norm=plt.Normalize(-1, 1))
sm.set_array([])
fig.colorbar(sm, cax=cbar_ax)

plt.tight_layout(rect=[0, 0, 0.9, 1])
plt.suptitle('Evolution of Conditional Correlation Matrix', fontsize=16, y=1.02)
plt.show()

In [ ]:
# Extract conditional volatilities from DCC model
conditional_volatilities = dcc_results.conditional_volatilities

# Convert to DataFrame for easier plotting
vol_df = pd.DataFrame(conditional_volatilities, index=returns_df.index, columns=returns_df.columns)

# Plot conditional volatilities
plt.figure(figsize=(14, 8))

for ticker in vol_df.columns:
    plt.plot(vol_df.index, vol_df[ticker], label=ticker)

plt.title('DCC-GARCH Conditional Volatilities')
plt.ylabel('Volatility (%)')
plt.xlabel('Date')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Compare DCC volatilities with rolling volatilities
plt.figure(figsize=(14, 8))

# Select a specific asset for comparison
comparison_asset = 'SPY'

# Plot both volatilities
plt.plot(vol_df.index, vol_df[comparison_asset], label='DCC-GARCH', linewidth=2)
plt.plot(rolling_vols.index, rolling_vols[comparison_asset], label=f'{window_size}-Day Rolling', 
         linewidth=2, alpha=0.7)

plt.title(f'Comparison of Volatility Estimates: {comparison_asset}')
plt.ylabel('Volatility (%)')
plt.xlabel('Date')
plt.legend()
plt.grid(True)
plt.show()

## 5. BEKK Model for Covariance Dynamics

Let's estimate a BEKK model, which directly models the covariance matrix dynamics.

In [ ]:
# For BEKK models, we'll use a subset of assets to reduce computational complexity
# BEKK models have many parameters and can be computationally intensive
subset_tickers = ['SPY', 'QQQ', 'XLF', 'XLE']
subset_returns = returns_df[subset_tickers].values

# Create and estimate a BEKK model
bekk_model = BEKK(len(subset_tickers),  # Number of assets
                 p=1, q=1,             # BEKK orders
                 distribution=Normal())

# Fit the model
print("Estimating BEKK model...")
bekk_results = bekk_model.fit(subset_returns)
print("BEKK estimation complete.")

# Display estimation results
print("
BEKK Estimation Results:")
print(f"Log-Likelihood: {bekk_results.log_likelihood:.4f}")
print(f"AIC: {bekk_results.aic:.4f}")
print(f"BIC: {bekk_results.bic:.4f}")
print(f"Number of Parameters: {len(bekk_results.parameters)}")

In [ ]:
# Extract conditional covariances from BEKK model
bekk_covariances = bekk_results.conditional_covariance

# Convert to conditional correlations for comparison
bekk_correlations = []
for cov_matrix in bekk_covariances:
    corr_matrix = cov2corr(cov_matrix)
    bekk_correlations.append(corr_matrix)

# Create a dictionary to store time series of pairwise correlations
bekk_pairwise_correlations = {}
for i, asset1 in enumerate(subset_tickers):
    for j, asset2 in enumerate(subset_tickers):
        if i < j:  # Only store unique pairs
            pair = (asset1, asset2)
            bekk_pairwise_correlations[pair] = [corr_matrix[i, j] for corr_matrix in bekk_correlations]

# Convert to DataFrame for easier plotting
bekk_corr_df = pd.DataFrame(bekk_pairwise_correlations, index=returns_df.index)

# Plot BEKK conditional correlations
plt.figure(figsize=(14, 8))

for pair in bekk_corr_df.columns:
    plt.plot(bekk_corr_df.index, bekk_corr_df[pair], label=f'{pair[0]} vs {pair[1]}')

plt.title('BEKK Conditional Correlations')
plt.ylabel('Correlation')
plt.xlabel('Date')
plt.legend()
plt.grid(True)
plt.ylim(-0.2, 1.0)  # Adjust as needed
plt.show()

In [ ]:
# Compare BEKK and DCC correlations
plt.figure(figsize=(14, 8))

# Select a specific pair for comparison
comparison_pair = ('SPY', 'XLF')  # S&P 500 vs Financials

# Get BEKK correlation for this pair
if comparison_pair in bekk_corr_df.columns:
    bekk_corr = bekk_corr_df[comparison_pair]
elif (comparison_pair[1], comparison_pair[0]) in bekk_corr_df.columns:  # Check reverse order
    bekk_corr = bekk_corr_df[(comparison_pair[1], comparison_pair[0])]
else:
    bekk_corr = None

# Get DCC correlation for this pair
if comparison_pair in corr_df.columns:
    dcc_corr = corr_df[comparison_pair]
elif (comparison_pair[1], comparison_pair[0]) in corr_df.columns:  # Check reverse order
    dcc_corr = corr_df[(comparison_pair[1], comparison_pair[0])]
else:
    dcc_corr = None

# Plot both correlations
if bekk_corr is not None:
    plt.plot(bekk_corr.index, bekk_corr, label='BEKK', linewidth=2)
if dcc_corr is not None:
    plt.plot(dcc_corr.index, dcc_corr, label='DCC-GARCH', linewidth=2, alpha=0.7)

plt.title(f'Comparison of Correlation Estimates: {comparison_pair[0]} vs {comparison_pair[1]}')
plt.ylabel('Correlation')
plt.xlabel('Date')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Extract conditional volatilities from BEKK model
bekk_volatilities = np.array([[np.sqrt(cov_matrix[i, i]) for i in range(len(subset_tickers))] 
                             for cov_matrix in bekk_covariances])

# Convert to DataFrame for easier plotting
bekk_vol_df = pd.DataFrame(bekk_volatilities, index=returns_df.index, columns=subset_tickers)

# Plot BEKK conditional volatilities
plt.figure(figsize=(14, 8))

for ticker in bekk_vol_df.columns:
    plt.plot(bekk_vol_df.index, bekk_vol_df[ticker], label=ticker)

plt.title('BEKK Conditional Volatilities')
plt.ylabel('Volatility (%)')
plt.xlabel('Date')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Compare BEKK and DCC volatilities
plt.figure(figsize=(14, 8))

# Select a specific asset for comparison
comparison_asset = 'SPY'

# Plot both volatilities
plt.plot(bekk_vol_df.index, bekk_vol_df[comparison_asset], label='BEKK', linewidth=2)
plt.plot(vol_df.index, vol_df[comparison_asset], label='DCC-GARCH', linewidth=2, alpha=0.7)

plt.title(f'Comparison of Volatility Estimates: {comparison_asset}')
plt.ylabel('Volatility (%)')
plt.xlabel('Date')
plt.legend()
plt.grid(True)
plt.show()

## 6. Constant Conditional Correlation (CCC) Model

Let's estimate a CCC model, which assumes constant correlations but time-varying volatilities.

In [ ]:
# Create and estimate a CCC model
ccc_model = CCC(returns_df.shape[1],  # Number of assets
               univariate_model=GARCH,  # Univariate model for each series
               univariate_params={'p': 1, 'q': 1},  # GARCH(1,1) for each series
               distribution=Normal())

# Fit the model
print("Estimating CCC model...")
ccc_results = ccc_model.fit(returns_array)
print("CCC estimation complete.")

# Display estimation results
print("
CCC Estimation Results:")
print(f"Log-Likelihood: {ccc_results.log_likelihood:.4f}")
print(f"AIC: {ccc_results.aic:.4f}")
print(f"BIC: {ccc_results.bic:.4f}")

# Display constant correlation matrix
print("
Constant Correlation Matrix:")
constant_corr = ccc_results.constant_correlation
constant_corr_df = pd.DataFrame(constant_corr, index=returns_df.columns, columns=returns_df.columns)
print(constant_corr_df)

In [ ]:
# Visualize the constant correlation matrix
plt.figure(figsize=(12, 10))
sns.heatmap(constant_corr_df, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0,
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
plt.title('CCC Model: Constant Correlation Matrix')
plt.tight_layout()
plt.show()

In [ ]:
# Extract conditional volatilities from CCC model
ccc_volatilities = ccc_results.conditional_volatilities

# Convert to DataFrame for easier plotting
ccc_vol_df = pd.DataFrame(ccc_volatilities, index=returns_df.index, columns=returns_df.columns)

# Plot CCC conditional volatilities
plt.figure(figsize=(14, 8))

for ticker in ccc_vol_df.columns:
    plt.plot(ccc_vol_df.index, ccc_vol_df[ticker], label=ticker)

plt.title('CCC Conditional Volatilities')
plt.ylabel('Volatility (%)')
plt.xlabel('Date')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Compare volatility estimates from different models
plt.figure(figsize=(14, 8))

# Select a specific asset for comparison
comparison_asset = 'SPY'

# Plot volatilities from different models
plt.plot(vol_df.index, vol_df[comparison_asset], label='DCC-GARCH', linewidth=2)
plt.plot(ccc_vol_df.index, ccc_vol_df[comparison_asset], label='CCC', linewidth=2, alpha=0.7)
if comparison_asset in bekk_vol_df.columns:
    plt.plot(bekk_vol_df.index, bekk_vol_df[comparison_asset], label='BEKK', linewidth=2, alpha=0.7)

plt.title(f'Comparison of Volatility Estimates: {comparison_asset}')
plt.ylabel('Volatility (%)')
plt.xlabel('Date')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Compare model fit using information criteria
model_comparison = pd.DataFrame({
    'Model': ['DCC-GARCH', 'BEKK', 'CCC'],
    'Log-Likelihood': [
        dcc_results.log_likelihood,
        bekk_results.log_likelihood,
        ccc_results.log_likelihood
    ],
    'AIC': [
        dcc_results.aic,
        bekk_results.aic,
        ccc_results.aic
    ],
    'BIC': [
        dcc_results.bic,
        bekk_results.bic,
        ccc_results.bic
    ],
    'Parameters': [
        len(dcc_results.parameters),
        len(bekk_results.parameters),
        len(ccc_results.parameters)
    ]
})

print("Model Comparison:")
model_comparison

## 7. Other Multivariate Models (OGARCH, RARCH, etc.)

Let's explore some other multivariate volatility models available in the MFE Toolbox.

In [ ]:
# Create and estimate an OGARCH model
# OGARCH uses principal component analysis to transform the data
ogarch_model = OGARCH(returns_df.shape[1],  # Number of assets
                     n_factors=3,          # Number of factors
                     univariate_model=GARCH,  # Univariate model for each factor
                     univariate_params={'p': 1, 'q': 1},  # GARCH(1,1) for each factor
                     distribution=Normal())

# Fit the model
print("Estimating OGARCH model...")
ogarch_results = ogarch_model.fit(returns_array)
print("OGARCH estimation complete.")

# Display estimation results
print("
OGARCH Estimation Results:")
print(f"Log-Likelihood: {ogarch_results.log_likelihood:.4f}")
print(f"AIC: {ogarch_results.aic:.4f}")
print(f"BIC: {ogarch_results.bic:.4f}")
print(f"Number of Parameters: {len(ogarch_results.parameters)}")

# Display factor loadings
print("
Factor Loadings:")
factor_loadings = ogarch_results.factor_loadings
factor_loadings_df = pd.DataFrame(factor_loadings, index=returns_df.columns, 
                                 columns=[f'Factor {i+1}' for i in range(factor_loadings.shape[1])])
print(factor_loadings_df)

In [ ]:
# Extract conditional covariances from OGARCH model
ogarch_covariances = ogarch_results.conditional_covariance

# Convert to conditional correlations for comparison
ogarch_correlations = []
for cov_matrix in ogarch_covariances:
    corr_matrix = cov2corr(cov_matrix)
    ogarch_correlations.append(corr_matrix)

# Create a dictionary to store time series of pairwise correlations
ogarch_pairwise_correlations = {}
for i, asset1 in enumerate(returns_df.columns):
    for j, asset2 in enumerate(returns_df.columns):
        if i < j:  # Only store unique pairs
            pair = (asset1, asset2)
            ogarch_pairwise_correlations[pair] = [corr_matrix[i, j] for corr_matrix in ogarch_correlations]

# Convert to DataFrame for easier plotting
ogarch_corr_df = pd.DataFrame(ogarch_pairwise_correlations, index=returns_df.index)

# Plot OGARCH conditional correlations for selected pairs
plt.figure(figsize=(14, 8))

for pair in pairs:
    if pair in ogarch_corr_df.columns:
        plt.plot(ogarch_corr_df.index, ogarch_corr_df[pair], label=f'{pair[0]} vs {pair[1]}')
    elif (pair[1], pair[0]) in ogarch_corr_df.columns:  # Check reverse order
        plt.plot(ogarch_corr_df.index, ogarch_corr_df[(pair[1], pair[0])], label=f'{pair[0]} vs {pair[1]}')

plt.title('OGARCH Conditional Correlations')
plt.ylabel('Correlation')
plt.xlabel('Date')
plt.legend()
plt.grid(True)
plt.ylim(-0.2, 1.0)  # Adjust as needed
plt.show()

In [ ]:
# Create and estimate a RiskMetrics model
# RiskMetrics uses exponential smoothing for covariance estimation
riskmetrics_model = RiskMetrics(returns_df.shape[1],  # Number of assets
                              lambda_=0.94)  # Decay factor

# Fit the model
print("Estimating RiskMetrics model...")
riskmetrics_results = riskmetrics_model.fit(returns_array)
print("RiskMetrics estimation complete.")

# Display estimation results
print("
RiskMetrics Estimation Results:")
print(f"Log-Likelihood: {riskmetrics_results.log_likelihood:.4f}")
print(f"AIC: {riskmetrics_results.aic:.4f}")
print(f"BIC: {riskmetrics_results.bic:.4f}")
print(f"Lambda: {riskmetrics_results.parameters[0]:.4f}")

In [ ]:
# Extract conditional covariances from RiskMetrics model
riskmetrics_covariances = riskmetrics_results.conditional_covariance

# Convert to conditional correlations for comparison
riskmetrics_correlations = []
for cov_matrix in riskmetrics_covariances:
    corr_matrix = cov2corr(cov_matrix)
    riskmetrics_correlations.append(corr_matrix)

# Create a dictionary to store time series of pairwise correlations
riskmetrics_pairwise_correlations = {}
for i, asset1 in enumerate(returns_df.columns):
    for j, asset2 in enumerate(returns_df.columns):
        if i < j:  # Only store unique pairs
            pair = (asset1, asset2)
            riskmetrics_pairwise_correlations[pair] = [corr_matrix[i, j] for corr_matrix in riskmetrics_correlations]

# Convert to DataFrame for easier plotting
riskmetrics_corr_df = pd.DataFrame(riskmetrics_pairwise_correlations, index=returns_df.index)

# Compare correlation estimates from different models
plt.figure(figsize=(14, 8))

# Select a specific pair for comparison
comparison_pair = ('SPY', 'XLF')  # S&P 500 vs Financials

# Get correlations for this pair from different models
models = {
    'DCC-GARCH': corr_df,
    'OGARCH': ogarch_corr_df,
    'RiskMetrics': riskmetrics_corr_df
}

for model_name, model_corr_df in models.items():
    if comparison_pair in model_corr_df.columns:
        model_corr = model_corr_df[comparison_pair]
    elif (comparison_pair[1], comparison_pair[0]) in model_corr_df.columns:  # Check reverse order
        model_corr = model_corr_df[(comparison_pair[1], comparison_pair[0])]
    else:
        model_corr = None
    
    if model_corr is not None:
        plt.plot(model_corr.index, model_corr, label=model_name, linewidth=2, alpha=0.7)

# Add constant correlation from CCC model
i = returns_df.columns.get_loc(comparison_pair[0])
j = returns_df.columns.get_loc(comparison_pair[1])
ccc_corr_value = constant_corr[i, j]
plt.axhline(y=ccc_corr_value, color='purple', linestyle='--', 
            label=f'CCC: {ccc_corr_value:.4f}', linewidth=2)

plt.title(f'Comparison of Correlation Estimates: {comparison_pair[0]} vs {comparison_pair[1]}')
plt.ylabel('Correlation')
plt.xlabel('Date')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Update model comparison with all models
model_comparison = pd.DataFrame({
    'Model': ['DCC-GARCH', 'BEKK', 'CCC', 'OGARCH', 'RiskMetrics'],
    'Log-Likelihood': [
        dcc_results.log_likelihood,
        bekk_results.log_likelihood,
        ccc_results.log_likelihood,
        ogarch_results.log_likelihood,
        riskmetrics_results.log_likelihood
    ],
    'AIC': [
        dcc_results.aic,
        bekk_results.aic,
        ccc_results.aic,
        ogarch_results.aic,
        riskmetrics_results.aic
    ],
    'BIC': [
        dcc_results.bic,
        bekk_results.bic,
        ccc_results.bic,
        ogarch_results.bic,
        riskmetrics_results.bic
    ],
    'Parameters': [
        len(dcc_results.parameters),
        len(bekk_results.parameters),
        len(ccc_results.parameters),
        len(ogarch_results.parameters),
        len(riskmetrics_results.parameters)
    ]
})

# Sort by AIC
model_comparison = model_comparison.sort_values('AIC')

print("Model Comparison:")
model_comparison

## 8. Portfolio Volatility Estimation and Forecasting

Let's use our multivariate models to estimate and forecast portfolio volatility.

In [ ]:
# Define portfolio weights
# We'll create an equal-weighted portfolio
n_assets = returns_df.shape[1]
equal_weights = np.ones(n_assets) / n_assets

# Create a DataFrame with weights
weights_df = pd.DataFrame({
    'Asset': returns_df.columns,
    'Weight': equal_weights
})

print("Portfolio Weights:")
weights_df

In [ ]:
# Calculate portfolio returns
portfolio_returns = returns_df.dot(equal_weights)

# Plot portfolio returns
plt.figure(figsize=(14, 6))
plt.plot(portfolio_returns.index, portfolio_returns, color='blue')
plt.title('Equal-Weighted Portfolio Returns')
plt.ylabel('Returns (%)')
plt.xlabel('Date')
plt.grid(True)
plt.show()

# Calculate rolling portfolio volatility
rolling_portfolio_vol = portfolio_returns.rolling(window=window_size).std() * np.sqrt(252)  # Annualized

# Plot rolling portfolio volatility
plt.figure(figsize=(14, 6))
plt.plot(rolling_portfolio_vol.index, rolling_portfolio_vol, color='red')
plt.title(f'{window_size}-Day Rolling Portfolio Volatility (Annualized)')
plt.ylabel('Volatility (%)')
plt.xlabel('Date')
plt.grid(True)
plt.show()

In [ ]:
# Calculate portfolio volatility using multivariate models
# For a portfolio with weights w, the variance is w' * Sigma * w
# where Sigma is the covariance matrix

# Function to calculate portfolio variance from covariance matrix
def portfolio_variance(cov_matrix, weights):
    return weights.T @ cov_matrix @ weights

# Calculate portfolio volatility from different models
portfolio_vols = {}

# DCC model
dcc_portfolio_var = np.array([portfolio_variance(cov_matrix, equal_weights) 
                             for cov_matrix in dcc_results.conditional_covariance])
portfolio_vols['DCC-GARCH'] = np.sqrt(dcc_portfolio_var) * np.sqrt(252)  # Annualized

# CCC model
ccc_portfolio_var = np.array([portfolio_variance(cov_matrix, equal_weights) 
                             for cov_matrix in ccc_results.conditional_covariance])
portfolio_vols['CCC'] = np.sqrt(ccc_portfolio_var) * np.sqrt(252)  # Annualized

# OGARCH model
ogarch_portfolio_var = np.array([portfolio_variance(cov_matrix, equal_weights) 
                               for cov_matrix in ogarch_results.conditional_covariance])
portfolio_vols['OGARCH'] = np.sqrt(ogarch_portfolio_var) * np.sqrt(252)  # Annualized

# RiskMetrics model
riskmetrics_portfolio_var = np.array([portfolio_variance(cov_matrix, equal_weights) 
                                    for cov_matrix in riskmetrics_results.conditional_covariance])
portfolio_vols['RiskMetrics'] = np.sqrt(riskmetrics_portfolio_var) * np.sqrt(252)  # Annualized

# Convert to DataFrame
portfolio_vols_df = pd.DataFrame(portfolio_vols, index=returns_df.index)

# Add rolling volatility for comparison
portfolio_vols_df['Rolling'] = rolling_portfolio_vol

# Plot portfolio volatility from different models
plt.figure(figsize=(14, 8))

for model in portfolio_vols_df.columns:
    plt.plot(portfolio_vols_df.index, portfolio_vols_df[model], label=model, linewidth=2, alpha=0.7)

plt.title('Portfolio Volatility Estimates from Different Models (Annualized)')
plt.ylabel('Volatility (%)')
plt.xlabel('Date')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Forecast portfolio volatility
# We'll use the best-performing model based on AIC
best_model_name = model_comparison.iloc[0]['Model']
print(f"Best model based on AIC: {best_model_name}")

# Select the corresponding model and results
if best_model_name == 'DCC-GARCH':
    best_model = dcc_model
    best_results = dcc_results
elif best_model_name == 'BEKK':
    best_model = bekk_model
    best_results = bekk_results
elif best_model_name == 'CCC':
    best_model = ccc_model
    best_results = ccc_results
elif best_model_name == 'OGARCH':
    best_model = ogarch_model
    best_results = ogarch_results
else:  # 'RiskMetrics'
    best_model = riskmetrics_model
    best_results = riskmetrics_results

# Set forecast horizon
forecast_horizon = 30  # 30 days ahead

# Generate forecasts
covariance_forecasts = best_model.forecast(
    returns_array,
    best_results.parameters,
    horizon=forecast_horizon,
    n_simulations=1000  # Number of Monte Carlo simulations
)

# Calculate portfolio variance forecasts
portfolio_var_forecasts = np.array([portfolio_variance(cov_matrix, equal_weights) 
                                  for cov_matrix in covariance_forecasts.mean])
portfolio_vol_forecasts = np.sqrt(portfolio_var_forecasts) * np.sqrt(252)  # Annualized

# Create forecast dates
last_date = returns_df.index[-1]
forecast_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=forecast_horizon, freq='B')

# Create DataFrame for forecasts
forecast_df = pd.DataFrame({
    'Volatility': portfolio_vol_forecasts
}, index=forecast_dates)

# Display forecast summary
print("
Portfolio Volatility Forecast Summary:")
forecast_df.head(10)

In [ ]:
# Plot portfolio volatility forecasts
plt.figure(figsize=(14, 8))

# Plot historical volatility
historical_vol = portfolio_vols_df[best_model_name]
plt.plot(historical_vol.index[-60:], historical_vol[-60:], label='Historical Volatility', color='blue')

# Plot forecasted volatility
plt.plot(forecast_df.index, forecast_df['Volatility'], label='Forecasted Volatility', color='red')

# Add vertical line to separate historical and forecasted periods
plt.axvline(x=returns_df.index[-1], color='black', linestyle='--', label='Forecast Start')

plt.title(f'Portfolio Volatility Forecast using {best_model_name}')
plt.ylabel('Volatility (% Annualized)')
plt.xlabel('Date')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Calculate Value-at-Risk (VaR) for the portfolio
# We'll use the forecasted volatility and assume normal distribution
from scipy import stats

# Define confidence levels
confidence_levels = [0.95, 0.99]

# Calculate VaR for each forecast horizon
var_df = pd.DataFrame(index=forecast_df.index)

for confidence in confidence_levels:
    # Calculate VaR assuming normal distribution
    # VaR = -μ - σ * z_α
    z_score = stats.norm.ppf(1 - confidence)
    daily_vol = forecast_df['Volatility'] / np.sqrt(252)  # Convert to daily
    var = -0 - daily_vol * z_score  # Assuming zero mean
    var_df[f'VaR {confidence*100:.0f}%'] = var

# Display VaR forecasts
print("Portfolio Value-at-Risk Forecasts:")
var_df.head(10)

In [ ]:
# Plot VaR forecasts
plt.figure(figsize=(14, 8))

for confidence in confidence_levels:
    plt.plot(var_df.index, var_df[f'VaR {confidence*100:.0f}%'], 
             label=f'VaR {confidence*100:.0f}%', linewidth=2)

plt.title('Portfolio Value-at-Risk Forecasts')
plt.ylabel('VaR (%)')
plt.xlabel('Date')
plt.legend()
plt.grid(True)
plt.show()

## 9. Asynchronous Processing for Computationally Intensive Models

For computationally intensive tasks like estimating multiple multivariate models, we can use asynchronous processing to improve efficiency.

In [ ]:
# Define an asynchronous function to estimate a multivariate model
async def estimate_model_async(model_class, model_name: str, returns: np.ndarray, 
                              model_params: dict) -> Tuple[str, Any]:
    """
    Asynchronously estimate a multivariate volatility model and return results.
    
    Parameters
    ----------
    model_class : class
        Multivariate model class (DCC, BEKK, etc.)
    model_name : str
        Name of the model for display purposes
    returns : np.ndarray
        Array of returns
    model_params : dict
        Dictionary of model parameters
        
    Returns
    -------
    Tuple[str, Any]
        Tuple containing model name and model results
    """
    print(f"Starting estimation of {model_name}...")
    
    try:
        # Create model
        model = model_class(**model_params)
        
        # Use fit_async if available, otherwise use regular fit
        if hasattr(model, 'fit_async'):
            results = await model.fit_async(returns)
        else:
            # Run in executor to avoid blocking
            loop = asyncio.get_event_loop()
            results = await loop.run_in_executor(None, lambda: model.fit(returns))
        
        print(f"Completed estimation of {model_name}")
        return model_name, results
    
    except Exception as e:
        print(f"Error estimating {model_name}: {e}")
        return model_name, None

In [ ]:
# Define an asynchronous function to estimate multiple models concurrently
async def estimate_multiple_models_async(returns: np.ndarray) -> Dict[str, Any]:
    """
    Asynchronously estimate multiple multivariate volatility models.
    
    Parameters
    ----------
    returns : np.ndarray
        Array of returns
        
    Returns
    -------
    Dict[str, Any]
        Dictionary mapping model names to results
    """
    # Define models to estimate
    n_assets = returns.shape[1]
    
    # Use a subset of assets for BEKK to reduce computational complexity
    subset_indices = [0, 1, 2, 3]  # First 4 assets
    subset_returns = returns[:, subset_indices]
    
    models_to_estimate = [
        # DCC model
        (DCC, "DCC-GARCH Normal", returns, {
            'n_assets': n_assets,
            'p': 1, 'q': 1,
            'univariate_model': GARCH,
            'univariate_params': {'p': 1, 'q': 1},
            'distribution': Normal()
        }),
        
        # DCC model with Student's t distribution
        (DCC, "DCC-GARCH Student's t", returns, {
            'n_assets': n_assets,
            'p': 1, 'q': 1,
            'univariate_model': GARCH,
            'univariate_params': {'p': 1, 'q': 1},
            'distribution': StudentT()
        }),
        
        # CCC model
        (CCC, "CCC Normal", returns, {
            'n_assets': n_assets,
            'univariate_model': GARCH,
            'univariate_params': {'p': 1, 'q': 1},
            'distribution': Normal()
        }),
        
        # BEKK model (using subset)
        (BEKK, "BEKK Normal", subset_returns, {
            'n_assets': len(subset_indices),
            'p': 1, 'q': 1,
            'distribution': Normal()
        }),
        
        # OGARCH model
        (OGARCH, "OGARCH Normal", returns, {
            'n_assets': n_assets,
            'n_factors': 3,
            'univariate_model': GARCH,
            'univariate_params': {'p': 1, 'q': 1},
            'distribution': Normal()
        }),
        
        # RiskMetrics model
        (RiskMetrics, "RiskMetrics", returns, {
            'n_assets': n_assets,
            'lambda_': 0.94
        })
    ]
    
    # Create tasks for each model
    tasks = [
        estimate_model_async(model_class, model_name, model_returns, model_params)
        for model_class, model_name, model_returns, model_params in models_to_estimate
    ]
    
    # Run tasks concurrently
    results = await asyncio.gather(*tasks)
    
    # Process results
    model_results = {}
    for model_name, result in results:
        if result is not None:
            model_results[model_name] = result
    
    return model_results


In [ ]:
# Run asynchronous estimation
async def main():
    print("Starting asynchronous model estimation...")
    start_time = datetime.datetime.now()
    
    # Estimate models
    model_results = await estimate_multiple_models_async(returns_array)
    
    # Calculate elapsed time
    end_time = datetime.datetime.now()
    elapsed_time = (end_time - start_time).total_seconds()
    
    print(f"
Estimated {len(model_results)} models in {elapsed_time:.2f} seconds")
    
    # Create comparison DataFrame
    comparison = []
    for model_name, result in model_results.items():
        comparison.append({
            'Model': model_name,
            'Log-Likelihood': result.log_likelihood,
            'AIC': result.aic,
            'BIC': result.bic,
            'Parameters': len(result.parameters)
        })
    
    comparison_df = pd.DataFrame(comparison)
    comparison_df = comparison_df.sort_values('AIC')
    
    print("
Model Comparison:")
    print(comparison_df)
    
    # Return results for further analysis
    return model_results, comparison_df

# Run the async function
async_results, async_comparison = await main()

In [ ]:
# Calculate portfolio volatility from asynchronously estimated models
async_portfolio_vols = {}

for model_name, result in async_results.items():
    # Skip BEKK model since it was estimated on a subset
    if 'BEKK' in model_name:
        continue
        
    # Calculate portfolio variance
    portfolio_var = np.array([portfolio_variance(cov_matrix, equal_weights) 
                             for cov_matrix in result.conditional_covariance])
    async_portfolio_vols[model_name] = np.sqrt(portfolio_var) * np.sqrt(252)  # Annualized

# Convert to DataFrame
async_portfolio_vols_df = pd.DataFrame(async_portfolio_vols, index=returns_df.index)

# Add rolling volatility for comparison
async_portfolio_vols_df['Rolling'] = rolling_portfolio_vol

# Plot portfolio volatility from different models
plt.figure(figsize=(14, 8))

for model in async_portfolio_vols_df.columns:
    plt.plot(async_portfolio_vols_df.index, async_portfolio_vols_df[model], label=model, linewidth=2, alpha=0.7)

plt.title('Portfolio Volatility Estimates from Different Models (Annualized)')
plt.ylabel('Volatility (%)')
plt.xlabel('Date')
plt.legend()
plt.grid(True)
plt.show()

## 10. Conclusion

In this notebook, we've demonstrated the use of multivariate volatility models in the MFE Toolbox. We've covered:

1. Data preparation and exploration for multiple assets
2. Correlation analysis and visualization
3. Dynamic Conditional Correlation (DCC) model estimation
4. BEKK model for covariance dynamics
5. Constant Conditional Correlation (CCC) model
6. Other multivariate models (OGARCH, RiskMetrics)
7. Portfolio volatility estimation and forecasting
8. Asynchronous processing for computationally intensive models

The MFE Toolbox provides a comprehensive set of multivariate volatility models for analyzing multiple asset returns, implemented using modern Python practices with NumPy, Pandas, and Numba acceleration. The class-based design with type hints and asynchronous processing capabilities makes it both powerful and user-friendly for complex multivariate analysis.

In [ ]:
# Display version information
print(f"MFE Toolbox version: {mfe.__version__}")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"Matplotlib version: {plt.matplotlib.__version__}")
print(f"Seaborn version: {sns.__version__}")